Install the required modules:

In [1]:
!pip install datasets
!pip install --upgrade gensim
!pip install langdetect
!pip install plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=d720646f80121d29e111777402284177aa4247ca67c6b695e5023e2b605162ff
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfu

Import the required modules:

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import random
import re
from datasets import load_dataset
from gensim.models.word2vec import Word2Vec
from langdetect import detect
from pandas.core.common import flatten
from sklearn.manifold import TSNE

Load the dataset:

In [3]:
dataset = load_dataset("timdettmers/openassistant-guanaco")

Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/9846 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/518 [00:00<?, ? examples/s]

In [4]:
docs = dataset['train']['text'] + dataset['test']['text']

def filter_english(strings):
  english_strings = []
  for string in strings:
    try:
      if detect(string) == 'en':
        english_strings.append(string)
    except:
      print(f"Error with {string}")
  return english_strings

docs = filter_english(docs)
docs[:5]

Error with ### Human: Generate me an ASCII of a person with a jetpack### Assistant: Here is an ASCII art representation of a person with a jetpack:

─────────▄███████████▄▄──────────────
──────▄██▀──────────▀▀██▄────────────
────▄█▀────────────────▀██───────────
──▄█▀────────────────────▀█▄─────────
─█▀──██──────────────██───▀██────────
█▀──────────────────────────██───────
█──███████████████████───────█───────
█────────────────────────────█───────
█────────────────────────────█───────
█────────────────────────────█───────
█────────────────────────────█───────
█────────────────────────────█───────
█▄───────────────────────────█───────
▀█▄─────────────────────────██───────
─▀█▄───────────────────────██────────
──▀█▄────────────────────▄█▀─────────
───▀█▄──────────────────██───────────
─────▀█▄──────────────▄█▀────────────
───────▀█▄▄▄──────▄▄▄███████▄▄───────
────────███████████████───▀██████▄───
─────▄███▀▀────────▀███▄──────█─███──
───▄███▄─────▄▄▄▄────███────▄▄████▀──
─▄███▓▓█─────█▓

['### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, lea

Let's separate the sentences written by the human from the ones of the assistant:

In [5]:
human_questions = []
assistant_answers = []

for string in docs:
  segments = string.split("### Human:")
  for segment in segments[1:]:
    #This if is added to add to the list of questions only those that have then received an answer
    if "### Assistant:" in segment:
      human_questions.append(segment.split("### Assistant:")[0].strip())

  segments = string.split("### Assistant:")
  for segment in segments[1:]:
      assistant_answers.append(segment.split("### Human:")[0].strip())

print(len(human_questions))
print(len(assistant_answers))

5497
5497


Let's clean the datasets and split them into different sentences:

In [6]:
human_questions = [re.sub('[\n\t]', '', doc) for doc in human_questions]
assistant_answers = [re.sub('[\n\t]', '', doc) for doc in assistant_answers]
human_questions = [re.sub('[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', '', doc) for doc in human_questions]
assistant_answers = [re.sub('[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', '', doc) for doc in assistant_answers]
human_questions = [re.split('[?!.]\s', doc) for doc in human_questions]
assistant_answers = [re.split('[?!.]\s', doc) for doc in assistant_answers]

Flatten the list of lists and tokenize each word:

In [7]:
human_questions = list(flatten(human_questions))
human_questions = [re.sub('\W', ' ', doc).lower().split() for doc in human_questions]

assistant_answers = list(flatten(assistant_answers))
assistant_answers = [re.sub('\W', ' ', doc).lower().split() for doc in assistant_answers]

#We also compute a combination of the two tet
full_text = human_questions + assistant_answers

 The minimum length of each sentence is 5, so we don't need to remove sentences that are composed by few words

 Now we train the Word2Vec on the human questions, by providing as input the tokenized words, the size of each embedding, the minimum number of occurences for each word and the context window size:



In [8]:
model_human = Word2Vec(human_questions, vector_size=30, min_count=5, window=10)

How many words do we have in our model?

In [9]:
len(model_human.wv)

2920

How is each embedding vector made?

In [10]:
term = 'house'
model_human.wv[term]

array([-0.10022627, -0.28386343,  0.33570367,  0.28759193,  0.13413888,
       -0.10477038,  0.547066  ,  0.25722325, -0.7903707 , -0.15968426,
        0.7472585 ,  0.2950165 ,  0.00574532, -0.38886628, -0.04450011,
       -0.16881196,  0.1716575 ,  0.13687074, -0.8949497 ,  0.16910127,
       -0.33417   , -0.23953557, -0.05624858,  0.734913  ,  0.41609782,
        0.64615244,  0.20772658,  0.502512  ,  0.57472485, -0.8008791 ],
      dtype=float32)

What are the most similar words to the word "short" ?

In [11]:
term = 'short'
model_human.wv.most_similar(term)

[('essay', 0.9964202046394348),
 ('an', 0.9956190586090088),
 ('outline', 0.9948337078094482),
 ('simple', 0.9948313236236572),
 ('detailed', 0.9941338896751404),
 ('script', 0.9939818978309631),
 ('summary', 0.993627667427063),
 ('generate', 0.993420422077179),
 ('story', 0.9933814406394958),
 ('overview', 0.9926937818527222)]

Now, let's do the same thing but with the assistant answers:

In [12]:
model_assistant = Word2Vec(assistant_answers, vector_size=30, min_count=5, window=10)

It's vocabulary size is more than 4 times the human one:

In [13]:
len(model_assistant.wv)

12121

Let's see an example of embedding:

In [14]:
term = 'house'
model_assistant.wv[term]

array([-0.48832294, -0.28287935,  0.7544729 ,  0.11766148,  0.11155996,
       -0.08016068,  0.4123599 , -0.04075557, -0.36732846,  0.01006183,
        0.8352033 , -0.26727426,  0.1843089 , -1.1807934 , -0.05451583,
       -0.33629668,  0.7343492 ,  0.61052924, -0.45716786,  0.16258717,
       -0.14574096, -0.6981791 , -0.11126268,  0.8815748 ,  0.25084248,
        1.0914153 , -0.11423443,  0.47439855,  0.60421103, -0.7637499 ],
      dtype=float32)

And the most similar words to the word "short":

In [15]:
term = 'short'
model_assistant.wv.most_similar(term)

[('christmas', 0.8631665110588074),
 ('reminder', 0.857660710811615),
 ('kids', 0.823764443397522),
 ('comedic', 0.8237086534500122),
 ('zoo', 0.8212695121765137),
 ('jung', 0.8203134536743164),
 ('frodo', 0.8160849213600159),
 ('rock', 0.8133822083473206),
 ('nietzsche', 0.8129684925079346),
 ('romantic', 0.8125600814819336)]

Finally, let's perform the same operations with the full_text to our disposal:

In [16]:
model_full = Word2Vec(full_text, vector_size=30, min_count=5, window=10)

The length of the vocabulary isn't much different from the one of just the assistant:

In [17]:
len(model_full.wv)

12963

Let's again an example of an embedding:

In [18]:
term = 'plant'
model_full.wv[term]

array([-0.5340554 ,  0.42246512,  1.9884299 ,  0.25283265, -0.6510467 ,
        0.11874328, -0.84897244,  0.20053248, -0.6813426 , -0.3740499 ,
       -0.10018178, -0.26812735,  0.3471495 ,  0.05615817, -0.1052966 ,
       -0.507851  ,  0.6414193 , -0.11641445, -0.41327474,  0.7297995 ,
        0.14223097,  0.21421175, -0.14000034,  0.7158812 ,  0.62412196,
        0.11308661,  0.34681618,  0.29757258,  0.67684865, -1.2338877 ],
      dtype=float32)

And once again the most similar word to the word short:

In [19]:
term = 'short'
model_full.wv.most_similar(term)

[('poem', 0.8259386420249939),
 ('joke', 0.8044517636299133),
 ('song', 0.7975432872772217),
 ('girl', 0.7951788902282715),
 ('great', 0.789358377456665),
 ('comedy', 0.7845564484596252),
 ('book', 0.7833995819091797),
 ('story', 0.7743746042251587),
 ('exciting', 0.7706369757652283),
 ('unease', 0.7466121315956116)]

Has the model understood the relation between this words?

In [20]:
vec = model_full.wv['king'] + (model_full.wv['woman'] - model_full.wv['man'])
vec

array([-0.02735159,  0.01934542,  0.7981969 ,  0.26874483, -1.0869787 ,
       -0.01829994,  0.33773643, -0.8313204 , -0.38398868,  0.02210933,
        0.60154533, -0.6529726 , -0.03031841, -0.52744126, -0.06677991,
       -0.85933304,  0.34845164,  0.70420575,  0.00219184, -0.14554876,
       -0.29848254, -0.2725364 ,  1.000072  ,  0.9581907 , -0.34120905,
        1.4137001 , -0.3794695 ,  0.8466449 , -0.14262947, -1.0663655 ],
      dtype=float32)

Seems it does not:

In [21]:
model_full.wv.most_similar(vec)

[('grace', 0.8986829519271851),
 ('agreed', 0.8963772654533386),
 ('pharaoh', 0.8908005952835083),
 ('bravery', 0.8876628279685974),
 ('decides', 0.8876504302024841),
 ('voldemort', 0.8862580060958862),
 ('realizes', 0.8816205859184265),
 ('odysseus', 0.8781604170799255),
 ('begins', 0.8724493384361267),
 ('remained', 0.872336208820343)]

And this kind of relation?

In [22]:
vec = model_full.wv['france'] + (model_full.wv['rome'] - model_full.wv['italy'])
vec

array([-0.32729685,  0.08111137,  1.5880258 , -0.3596708 , -1.3201766 ,
        0.17518401,  0.2744375 , -0.5860498 , -0.39076585, -0.65660185,
        0.36202615, -0.41683426,  0.1911022 , -0.3547143 , -0.75112635,
       -1.5663768 , -0.18569042,  0.9215853 , -1.1430662 ,  0.4271389 ,
       -0.88653827,  0.13474075,  0.9819212 ,  0.8636242 ,  0.0335204 ,
        1.3094158 , -0.74908495,  0.80793154,  0.22818908, -1.2540319 ],
      dtype=float32)

It seems to have a better idea of what we are talking about, but still he hasn't fully understood the relation:

In [23]:
model_full.wv.most_similar(vec)

[('france', 0.9804287552833557),
 ('rome', 0.9796115756034851),
 ('late', 0.9538754224777222),
 ('independence', 0.9420555233955383),
 ('death', 0.94028639793396),
 ('roman', 0.9383842945098877),
 ('marcus', 0.9381493926048279),
 ('elizabeth', 0.9362397789955139),
 ('jesus', 0.9343924522399902),
 ('british', 0.9339328408241272)]

#Visualizing the embedding vector using t-SNE

We'll now produce a graphical representation of a subset of the embeddings, because to reduce the time required for computation, we'll limit our representation to 500 random samples:

In [24]:
random_samples_human = random.sample(list(model_human.wv.key_to_index), 500)
word_vectors_human = model_human.wv[random_samples_human]

random_samples_assistant = random.sample(list(model_assistant.wv.key_to_index), 500)
word_vectors_assistant = model_assistant.wv[random_samples_assistant]

random_samples_full = random.sample(list(model_full.wv.key_to_index), 500)
word_vectors_full = model_full.wv[random_samples_full]

We'll provide the vectors to the TSNE algorithm, to fit a model and have a 3 dimensional representation:

In [25]:
tsne = TSNE(n_components=3, n_iter=2000)

tsne_embeddings_human = tsne.fit_transform(word_vectors_human)
tsne_embeddings_assistant = tsne.fit_transform(word_vectors_assistant)
tsne_embeddings_full = tsne.fit_transform(word_vectors_full)

We transpose the matrix, as to have each dimension in each row of the resulting matrix and we get the coordinates of each point:

In [26]:
x_human, y_human, z_human = np.transpose(tsne_embeddings_human)
x_assistant, y_assistant, z_assistant = np.transpose(tsne_embeddings_assistant)
x_full, y_full, z_full = np.transpose(tsne_embeddings_full)

We draw the plots (we also reduced the amount of samples to be shown for a bettere representation):

In [27]:
fig = px.scatter_3d(x=x_human[:150], y=y_human[:150], z=z_human[:150], text=random_samples_human[:150])
fig.update_traces(marker=dict(size=3,line=dict(width=2)))
fig.show()

In [28]:
fig = px.scatter_3d(x=x_assistant[:150], y=y_assistant[:150], z=z_assistant[:150], text=random_samples_assistant[:150])
fig.update_traces(marker=dict(size=3,line=dict(width=2)))
fig.show()

In [29]:
fig = px.scatter_3d(x=x_full[:150], y=y_full[:150], z=z_full[:150], text=random_samples_full[:150])
fig.update_traces(marker=dict(size=3,line=dict(width=2)))
fig.show()